In [12]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

from conllu_to_df import process_file
from extract_features import extract_features
import numpy as np

In [2]:
train_file = "../UP-1.0/input/en_ewt-up-train.conllu"
dev_file = "../UP-1.0/input/en_ewt-up-dev.conllu"
test_file = "../UP-1.0/input/en_ewt-up-test.conllu"

out_train_file = '../UP-1.0/output/train.csv'
out_test_file = '../UP-1.0/output/test.csv'
out_dev_file = '../UP-1.0/output/dev.csv'

feat_train_file = out_train_file.replace('output', 'features')
feat_test_file = out_test_file.replace('output', 'features')
feat_dev_file = out_dev_file.replace('output', 'features')

In [3]:
# process_file(dev_file, out_dev_file)
# process_file(test_file, out_test_file)
# process_file(train_file, out_train_file)

In [8]:
extract_features(out_dev_file, feat_dev_file)
extract_features(out_train_file, feat_train_file)
extract_features(out_test_file, feat_test_file)

exctacted voice
exctacted voice
exctacted voice


In [5]:
vectorizer = DictVectorizer()

In [ ]:
def extract_features_and_labels(trainingfile):
    features = []
    gold = []
    with open(trainingfile, 'r', encoding='utf-8') as infile:
        for line in infile:
            components = line.rstrip('\n').split('\t')
            if len(components) > 0:
                token = components[0]
                pos = components[1]
                lemma = components[2]
                dependency = components[3]
                head = components[4]
                head_pos = components[5]
                morph_feat = components[6]
                ent_type = components[7]
                ent_dep = components[8]
                voice = components[9]
                gold_a = components[-1]
                feature_dict = {'Token': token,
                                'PoS': pos,
                                'Lemma': lemma, 
                                'dependency': dependency, 
                                'head': head, 
                                'head pos': head_pos,
                                'morph feature': morph_feat,
                                'entity type': ent_type,
                                'entity dep': ent_dep,
                                'voice': voice}
                features.append(feature_dict)
                gold.append(gold_a)
    return features, gold

In [31]:
train_df = pd.read_csv(feat_train_file, sep='\t', low_memory=False)
train_data = train_df.to_dict('records')
del train_df
train_vectorized = vectorizer.fit_transform(train_data)


In [30]:
dev_df = pd.read_csv(feat_dev_file, sep='\t', low_memory=False)
dev_data = dev_df.to_dict('records')
del dev_df
dev_vectorized = vectorizer.transform(dev_data)

test_df = pd.read_csv(feat_test_file, sep='\t', low_memory=False)
test_data = test_df.to_dict('records')
del test_df
test_vectorized = vectorizer.transform(test_data)

In [33]:
train_vectorized.shape
dummy = np.ones(train_vectorized.shape[0])

0.0

In [21]:
train_vectorized = np.array(train_vectorized)
dev_vectorized = np.array(dev_vectorized)
test_vectorized = np.array(test_vectorized)

In [22]:
X = train_vectorized.data[:, :-1]
X_dev = dev_vectorized.data[:, :-1]

y = train_vectorized[:, -1]
y_dev = dev_vectorized[:, -1]

IndexError: too many indices for array: array is 0-dimensional, but 2 were indexed

In [26]:
train_vectorized.data.shape

()

In [ ]:
model = LogisticRegression().fit(X, y)

In [ ]:
y_pred = model.predict(X_dev)

In [ ]:
print("METRICS")
print('precision', precision_score(y_dev, y_pred), sep='\t')
print('recall', recall_score(y_dev, y_pred), sep='\t')
print('f1', f1_score(y_dev, y_pred), sep='\t')